In [1]:
#coding=utf8
"""
# Author : Jianbai(Gus) Ye
# created at Feb 2 2019
# pytorch implementation of HSIC bottleneck method
# reference : https://github.com/forin-xyz/Keras-HSIC-Bottleneck
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
from __future__ import unicode_literals
import torch
from torch import nn, optim
import numpy as np
import sys
import matplotlib.pyplot as plt
from torchsummary import summary
# from collections import Iterable
import matplotlib.pyplot as plt

import torch.nn.functional as F
import time
from utils import *

In [2]:
torch.manual_seed(1)
device = "cuda:1"
batch_size = 256
train_loader, test_loader = load_data(batch_size=batch_size)

In [3]:
class Block(nn.Module):
    def __init__(self, inplane, outplane):
        super(Block, self).__init__()
        self.linear = nn.Linear(inplane, outplane)
        self.bn = nn.BatchNorm1d(outplane, affine=False)
        self.act = nn.GELU()
    def forward(self, x):
        x = self.linear(x)
        x = self.act(x)
        x = self.bn(x)
        return x
        
class Test_model(nn.Module):
    def __init__(self):
        super(Test_model, self).__init__()
        
        self.units = [784, 256, 128, 128]
#         self.module_list = [Block(self.units[i], self.units[i+1]).to(device) for i in range(len(self.units)-1)]
        self.module_list = nn.ModuleList( [Block(self.units[i], self.units[i+1]) for i in range(len(self.units)-1)])
        
        self.f3 = nn.Dropout(p=0.2)
        self.output_layer  = nn.Linear(self.units[-1], 10)        
        self.act2 = nn.ReLU()
        
    def forward(self, data):
        x = data
        output = []
        for module in self.module_list:
            x_ = module(x.detach())
            x = module(x)
            output.append(x_)
        x = self.f3(x)
        x_ = self.act2(self.output_layer(x.detach()))
        x = self.act2(self.output_layer(x))
        output.append(x_)
        return x, output
    
class HSICBottleneck:
    def __init__(self, model, batch_size, lambda_0, sigma, multi_sigma=None,lr=0.01):
        self.model      = model
        self.batch_size = batch_size
        self.lambda_0   = lambda_0
        self.sigma      = sigma
        self.extractor  = 'hsic'
        self.last_linear = "output_layer"
        self.lr         = lr
        self.multi_sigma = multi_sigma
        assert isinstance(self.multi_sigma, Iterable) if  multi_sigma is not None else True
        
        self.opt = optim.AdamW(self.model.parameters(), lr=0.001)
#         self.opt = optim.SGD(self.model.parameters(), lr=0.001)
        self.track_loss1 = []
        self.track_loss2 = []
        self.track_loss3 = []
        
        self.loss = "CE"
        if self.loss == "mse":
            self.output_criterion = nn.MSELoss()
        elif self.loss == "CE":
            self.output_criterion = nn.CrossEntropyLoss()
        
    def step(self, input_data, labels):
        
        one_hot_labels = F.one_hot(labels, num_classes=10)
        if self.loss == "mse":
            labels = F.one_hot(labels, num_classes=10).float()
        
        Kx  = kernel_matrix(input_data, self.sigma)
        Ky = kernel_matrix(one_hot_labels, self.sigma)
        
        total_loss1 = 0.
        total_loss2 = 0.
        total_loss3 = 0.
        y_pred, hidden_zs = self.model(input_data)
        
        kernel_list = list()
        for num, feature in enumerate(hidden_zs):
            kernel_list.append(kernel_student(feature, self.sigma))
        
        for num, feature in enumerate(kernel_list):
            if num == (len(hidden_zs)-1): 
                loss1 = norm_HSIC(feature, kernel_list[num-1], self.batch_size, device)
#                 loss2 = - self.lambda_0*norm_HSIC(feature, kernel_list[num+1], self.batch_size, device)
                total_loss3 += self.output_criterion(feature, labels)
            elif num == 0:
                loss1 = norm_HSIC(feature, Kx, self.batch_size, device)
                loss2 = - self.lambda_0*norm_HSIC(feature, Ky, self.batch_size, device)
            else:
                loss1 = norm_HSIC(feature, kernel_list[num-1], self.batch_size, device)
                loss2 = - self.lambda_0*norm_HSIC(feature, Ky, self.batch_size, device)
            total_loss1 += loss1
            total_loss2 += loss2
                
#         total_loss = total_loss1 + total_loss2 + total_loss3
        total_loss = total_loss2 + total_loss3
        self.opt.zero_grad()
        total_loss.backward()
        self.opt.step()
        
#         self.track_loss1.append(total_loss1.item())
        self.track_loss2.append(total_loss2.item())
        self.track_loss3.append(total_loss3.item())
                
        return total_loss1.item(), total_loss2.item(), total_loss3.item()
    
    def tune_output(self, input_data, labels):
        
        if self.loss == "mse":
            one_hot_labels = F.one_hot(labels, num_classes=10)
            labels = F.one_hot(labels, num_classes=10).float()
        
        y_pred, hidden_zs = self.model(input_data)
        total_loss = self.output_criterion(hidden_zs[-1], labels)
        self.opt.zero_grad()
        total_loss.backward()
        self.opt.step()
    
        return total_loss.item()
    
def show_result():
    model.eval()
    with torch.no_grad():
        counts, correct, counts2, correct2 = 0, 0, 0, 0        
        for batch_idx, (data, target) in enumerate(train_loader): 
            output = hsic.model.forward(data.view(batch_size, -1).to(device))[0].cpu()
            pred = output.argmax(dim=1, keepdim=True)
            correct += (pred[:,0] == target).float().sum()
            counts += len(pred)
        for batch_idx, (data, target) in enumerate(test_loader): 
            output = hsic.model.forward(data.view(batch_size, -1).to(device))[0].cpu()
            pred = output.argmax(dim=1, keepdim=True)
            correct2 += (pred[:,0] == target).float().sum()
            counts2 += len(pred)
        print("Training  ACC: {:.2f} \t Testing ACC: {:.2f}".format(correct/counts, correct2/counts2))

In [ ]:


for sigma in [0.5, 1, 3, 10]:
    for lambda_0 in [100, 300, 1000, 3000]:
        print("===============================")
        print("Sigma {} Lambda {}".format(sigma, lambda_0))
        
        model = Test_model()
        model.to(device)
        model.train()
        HSIC_epochs = 100
#         lambda_0 = 500

        hsic = HSICBottleneck(model, batch_size=batch_size, lambda_0=lambda_0, sigma=sigma)

        start = time.time()
        for epoch in range(HSIC_epochs):
            model.train()

            total_loss1, total_loss2, total_loss3, total_loss_tune = 0, 0, 0, 0
            for batch_idx, (data, target) in enumerate(train_loader):
                data = data.view(batch_size, -1)
                loss1, loss2, loss3 = hsic.step(data.view(batch_size, -1).to(device), target.to(device))
                total_loss_tune += hsic.tune_output(data.view(batch_size, -1).to(device), target.to(device))
                total_loss1 += loss1
                total_loss2 += loss2
                total_loss3 += loss3
            if epoch in range(0, 100, 10):
                
                print("EPOCH %d" % epoch)
                model.eval()
                show_result()
                sys.stdout.write("{:.3f}, {:.3f}, {:.3f}, {:.3f}".format(total_loss1/(batch_idx+1), 
                                                                         total_loss2/lambda_0*100/(batch_idx+1), 
                                                                         total_loss3/(batch_idx+1),
                                                                         total_loss_tune/(batch_idx+1)))
                sys.stdout.flush()
                sys.stdout.write('\n')
                print("{:.2f}".format(time.time()-start))
                start = time.time()

Sigma 0.5 Lambda 100
EPOCH 0
Training  ACC: 0.95 	 Testing ACC: 0.94
0.015, -0.055, 5.562, 0.520
10.27
EPOCH 10
Training  ACC: 0.97 	 Testing ACC: 0.97
0.015, -0.055, 5.553, 0.106
76.55
EPOCH 20
Training  ACC: 0.98 	 Testing ACC: 0.97
0.015, -0.055, 5.552, 0.093
76.60
EPOCH 30
Training  ACC: 0.98 	 Testing ACC: 0.97
0.015, -0.055, 5.548, 0.085
76.40
EPOCH 40
Training  ACC: 0.98 	 Testing ACC: 0.97
0.015, -0.055, 5.551, 0.077
76.73
EPOCH 50
Training  ACC: 0.98 	 Testing ACC: 0.97
0.015, -0.055, 5.550, 0.077
76.72
EPOCH 60
Training  ACC: 0.98 	 Testing ACC: 0.97
0.015, -0.055, 5.551, 0.070
76.71
EPOCH 70
Training  ACC: 0.98 	 Testing ACC: 0.97
0.015, -0.055, 5.551, 0.071
76.65
EPOCH 80
Training  ACC: 0.98 	 Testing ACC: 0.97
0.015, -0.055, 5.553, 0.071
76.86
EPOCH 90
Training  ACC: 0.98 	 Testing ACC: 0.97
0.015, -0.055, 5.553, 0.068
76.79
Sigma 0.5 Lambda 300
EPOCH 0
Training  ACC: 0.95 	 Testing ACC: 0.95
0.015, -0.055, 5.558, 0.563
9.79
EPOCH 10
Training  ACC: 0.98 	 Testing ACC: 0.97

In [ ]:
from sklearn.decomposition import PCA 
output, hidden = hsic.model(data.view(batch_size, -1).to(device))
t = F.one_hot(target, num_classes=10).float()

for i, z in enumerate(hidden):
    print(i)
    z_ = z.detach().cpu()
    fig, axes = plt.subplots(1,3,figsize=(12,4))
    axes[0].imshow(mean(kernel_matrix(data.view(batch_size, -1), 100)))
    axes[1].imshow(mean(kernel_matrix(z_, 100)))
    axes[2].imshow(mean(kernel_matrix(t, 100)))
    plt.pause(0.1)
    
    fig, axes = plt.subplots(1,3,figsize=(12,4))
    axes[0].imshow(mean(kernel_student(data.view(batch_size, -1), 100)))
    axes[1].imshow(mean(kernel_student(z_, 100)))
    axes[2].imshow(mean(kernel_student(t, 100)))
    plt.pause(0.1)

In [ ]:
target